In [1]:
import re
import os
import tkinter as tk
from tkinter import ttk
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import simpledialog
import smtplib
from email.mime.text import MIMEText

# VERIFICATION PAGE Works Well by 18/08/2023 by 23:40
class VerificationPage:  # <-- Check the indentation of this line
    def __init__(self, parent, go_next):
        self.parent = parent
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        ttk.Label(self.frame, text="Welcome to Ashcroft_Johnson's Stroke Diagnosis and Prediction Dashboard",
                  font=("Helvetica", 18, "bold"), background="light blue").pack(pady=20)

        ttk.Label(self.frame, text="VERIFICATION PAGE",
                  font=("Helvetica", 14, "bold"), background="light blue").pack()
        
        self.checkbox_var = tk.BooleanVar(value=False)
        self.login_button = ttk.Button(self.frame, text="LOGIN", command=go_next, state="disabled", style="Green.TButton")
        self.login_button.pack(side="left", padx=10)

        # Added this command@14:12 by 18/08/2023
        # Use self.go_to_registration_page as the command for the REGISTER button
        self.register_button = ttk.Button(self.frame, text="REGISTER", command=self.go_to_registration_page, style="Green.TButton")
        self.register_button.pack(side="right", padx=10)
        
        ttk.Label(self.frame, text="New User? Register to use the application").pack()

        ttk.Checkbutton(self.frame, text="Already have an account, please login", variable=self.checkbox_var,
                        command=self.update_login_button_state).pack()

    def update_login_button_state(self):
        if self.checkbox_var.get():
            self.login_button.state(["!disabled"])
        else:
            self.login_button.state(["disabled"])
            
    def go_to_registration_page(self):
        self.frame.destroy()  # Destroy the current frame
        RegistrationPage(self.parent, self.go_next, self.go_to_registration_page)  # Create a RegistrationPage instance

    # Other methods...
# added just now 
    def go_next(self):
        # Implement the logic to proceed to the next page
        pass
        
        
        
# LOGIN PAGE OKAY, Works Well by 18/08/2023 by 23:43
class LoginPage:
    def __init__(self, parent, go_next):
        self.parent = parent
        self.go_next = go_next
        self.attempts = 0  # Initialise the attempts counter
        
        # Added now 19/08/2023
        self.go_to_data_protection = go_to_data_protection
        
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        ttk.Label(self.frame, text="Ashcroft_Johnson's Stroke Diagnosis and Prediction Dashboard",
                  font=("Helvetica", 18, "bold"), background="light blue").pack(pady=20)
        ttk.Label(self.frame, text="Login Page", font=("Helvetica", 24, "bold")).pack(pady=(50, 20))
        
        ttk.Label(self.frame, text="Email Address:", font=("Helvetica", 12)).pack()
        self.username_entry = ttk.Entry(self.frame, font=("Helvetica", 12))
        self.username_entry.pack(padx=20, pady=5, fill="x")
        
        ttk.Label(self.frame, text="Password:", font=("Helvetica", 12)).pack()
        self.password_entry = ttk.Entry(self.frame, show="*", font=("Helvetica", 12))
        self.password_entry.pack(padx=20, pady=5, fill="x")
        
        self.checkbox_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(self.frame, text="Remember me", variable=self.checkbox_var).pack(pady=(10, 20))
        
        self.login_button = ttk.Button(self.frame, text="Login", command=self.perform_login, style="Green.TButton")
        self.login_button.pack(pady=(0, 20), padx=20, fill="x")
        
        ttk.Label(self.frame, text="New User? Register to use the application", font=("Helvetica", 10)).pack()

        self.status_label = ttk.Label(self.frame, text="", font=("Helvetica", 12))
        self.status_label.pack()
        
        self.forgot_password_label = ttk.Label(self.frame, text="Forgot Password?", foreground="blue", cursor="hand2", font=("Helvetica", 12))
        self.forgot_password_label.pack()
        self.forgot_password_label.bind("<Button-1>", self.open_forgot_password_link)
        
        self.forgot_email_label = ttk.Label(self.frame, text="Forgot Email Address?", foreground="blue", cursor="hand2", font=("Helvetica", 12))
        self.forgot_email_label.pack()
        self.forgot_email_label.bind("<Button-1>", self.open_forgot_email_link)

    def open_forgot_email_link(self, event):
        self.parent.withdraw()  # Hide the main window
        self.forgot_email_window = tk.Toplevel()
        self.forgot_email_window.title("Forgot Email Address Recovery")
        
        ttk.Label(self.forgot_email_window, text="Forgot Email Address Recovery", font=("Helvetica", 16, "bold")).pack(pady=20)
        
        ttk.Label(self.forgot_email_window, text="Please contact the system administrator to recover your email address.",
                  font=("Helvetica", 12)).pack(pady=20)
        
        ttk.Button(self.forgot_email_window, text="Close", command=self.close_forgot_email_window).pack(pady=10)
        
    def close_forgot_email_window(self):
        self.forgot_email_window.destroy()
        self.parent.deiconify()  # Show the main window again
        

    def perform_login(self):
        email = self.username_entry.get()
        password = self.password_entry.get()
        
        if email == "admin" and password == "password":
            self.status_label.config(text="Login successful", foreground="green")
            self.attempts = 0  # Reset the attempts counter on successful login
        else:
            self.attempts += 1
            self.status_label.config(text="Login failed", foreground="red")
            
            if self.attempts >= 3:
                self.block_account()
                
    def block_account(self):
        messagebox.showerror("Account Blocked", "You have entered incorrect credentials for the third time.\nYour account is blocked; Contact the Administrator.")
        self.parent.quit()  # Quit the application after blocking the account

            
    def open_forgot_password_link(self, event):
        email = simpledialog.askstring("Forgot Password", "Enter your email address:")
        if email:
            if self.send_password_reset_email(email):
                messagebox.showinfo("Forgot Password", f"Password reset link has been sent to {email}")
            else:
                messagebox.showerror("Error", "Failed to send password reset to the email provided.")
        
    def send_password_reset_email(self, email):
        try:
            smtp_server = "smtp.gmail.com"
            smtp_port = 465
            sender_email = "ashcroft.j.johnson@gmail.com"
            sender_password = "cpwmuzivjihmzfhx"
            
            server = smtplib.SMTP_SSL(smtp_server, smtp_port)
            server.login(sender_email, sender_password)
            
            subject = "Password Reset"
            message = "Click the link to reset your password: https://email_address/reset_password"
            email_content = MIMEText(message)
            email_content["Subject"] = subject
            email_content["From"] = sender_email
            email_content["To"] = email
            
            server.sendmail(sender_email, [email], email_content.as_string())
            server.quit()
            return True
        
        except Exception as e:
            print("Error sending email:", e)
            return False

        
 # REGISTRATION PAGE Works Well @18/08/2023 by  23:43   
class RegistrationPage:
    def __init__(self, parent, go_next, go_back):
        self.parent = parent
        self.go_next = go_next
        self.go_back = go_back
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)

        ttk.Label(self.frame, text="Ashcroft_Johnson's Stroke Diagnosis and Prediction Dashboard",
               font=("Helvetica", 14, "bold"), background="light blue").pack(pady=10)
        ttk.Label(self.frame, text="Registration Page",
                  font=("Helvetica", 14, "bold")).pack(pady=(10, 5))
        
        self.username_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.nhs_number_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.full_name_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.age_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.gender_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.email_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.mobile_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.password_entry = ttk.Entry(self.frame, show="*", font=("Helvetica", 10))
        self.re_password_entry = ttk.Entry(self.frame, show="*", font=("Helvetica", 10))
        
        labels_and_entries = [
            ("Username:", self.username_entry),
            ("Enter Your NHS Confidential Number:", self.nhs_number_entry),
            ("Your Full Name:", self.full_name_entry),
            ("Age:", self.age_entry),
            ("Gender:", self.gender_entry),
            ("Email Address:", self.email_entry),
            ("Enter Your Mobile Number:", self.mobile_entry),
            ("Create Password:", self.password_entry),
            ("Re-enter Password:", self.re_password_entry)
        ]
        
        for label_text, entry_widget in labels_and_entries:
            label = ttk.Label(self.frame, text=label_text, font=("Helvetica", 12), anchor="w")
            label.pack(padx=10, pady=2, fill="x")
            entry_widget.pack(padx=10, pady=2, fill="x")
            
        ttk.Button(self.frame, text="Create Account", command=self.create_account, style="Green.TButton").pack(pady=10, padx=10, fill="x")
        ttk.Button(self.frame, text="Back", command=self.go_back).pack(pady=5, padx=10, fill="x")

    def create_account(self):
        # Implement the logic to create the user account
        # After creating the account, show a popup message
        messagebox.showinfo("Account Created", "Your account has been successfully created!")
        
        # Navigate to the Login Page
        self.parent.withdraw()  # Hide the current window
        login_page = LoginPage(self.parent, self.go_next)
        
        
class DataProtectionPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
    def __init__(self, parent, go_next, go_back):
        self.parent = parent
        self.go_next = go_next
        self.go_back = go_back
        
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        ttk.Label(self.frame, text="Data Protection Page", font=("Helvetica", 18, "bold")).pack(pady=20)
        
        ttk.Label(self.frame, text="Data Protection Act 2018 in the UK", font=("Helvetica", 14)).pack(pady=(0, 10))
        
        self.checkbox1_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(self.frame, text="I Am aware of the Data Protection Act.", variable=self.checkbox1_var).pack()
        
        self.checkbox2_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(self.frame, text="I accept all guidance of the Data Protection Act 2018.", variable=self.checkbox2_var).pack()

        ttk.Button(self.frame, text="PROCEED", command=self.proceed).pack(pady=(20, 0))
        ttk.Button(self.frame, text="DECLINE", command=self.decline).pack(pady=(10, 0))

    def proceed(self):
        if self.checkbox1_var.get() and self.checkbox2_var.get():
            self.parent.withdraw()  # Hide the current window
            home_page = HomePage(self.parent, self.go_next)

    def decline(self):
        response = messagebox.askquestion("Confirm Decline", "Are you sure you want to decline?")
        if response == "yes":
            self.parent.quit()  # Quit the application
            

# ... (Previous code)

class LoginPage:
    def __init__(self, parent, go_next, go_to_data_protection):
        self.parent = parent
        self.go_next = go_next
        self.go_to_data_protection = go_to_data_protection
        # ... (Rest of the code)

    def open_data_protection(self):
        self.parent.withdraw()  # Hide the main window
        data_protection_page = DataProtectionPage(self.parent, self.go_next, self.go_to_data_protection)     

class HomePage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)

        
class SystemAdministratorAuthorisationPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class UsersPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class PredictionPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class AnalysisPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class SystemAdminEditingPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        
class ContactUsPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class StrokePredictionDashboard:
    def __init__(self, root):
        self.root = root
        self.root.title("Stroke Diagnosis and Prediction Dashboard")

        self.pages = [
            VerificationPage, LoginPage, RegistrationPage, 
            DataProtectionPage, HomePage, SystemAdministratorAuthorisationPage,
            UsersPage, PredictionPage, AnalysisPage,SystemAdminEditingPage,
            ContactUsPage# Add other pages here
        ]

        self.current_page_index = 0

        self.back_button = ttk.Button(root, text="Back", command=self.go_back)
        self.update_buttons()

        self.show_page(self.current_page_index)

    def update_buttons(self):
        if self.current_page_index == 0:
            self.back_button.pack_forget()
        else:
            self.back_button.pack(side="left")

    def show_page(self, index):
        if 0 <= index < len(self.pages):
            if hasattr(self, "current_page"):
                self.current_page.frame.pack_forget()

            self.current_page = self.pages[index](self.root, self.go_next)
            self.update_buttons()

    def go_back(self):
        if self.current_page_index > 0:
            self.current_page_index -= 1
            self.show_page(self.current_page_index)

    def go_next(self):
        if self.current_page_index < len(self.pages) - 1:
            self.current_page_index += 1
            self.show_page(self.current_page_index)

if __name__ == "__main__":
    root = tk.Tk()
    
     # Added now 18/08/2023
    # Create a custom style
    style = ttk.Style(root)
    style.configure("Green.TButton", background="green")
    
    
     # Set the screen size to 13 inches (adjust the width and height values as needed)
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    inches_per_mm = 0.0393701
    desired_screen_size_inches = 20
    desired_width = int(screen_width * inches_per_mm * desired_screen_size_inches)
    desired_height = int(screen_height * inches_per_mm * desired_screen_size_inches)
    root.geometry(f"{desired_width}x{desired_height}")
    
    app = StrokePredictionDashboard(root)
    root.mainloop()

# Copy To Save 19/08/2023

In [ ]:
import re
import os
import tkinter as tk
from tkinter import ttk
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import simpledialog
import smtplib
from email.mime.text import MIMEText

# VERIFICATION PAGE Works Well by 18/08/2023 by 23:40
class VerificationPage:  # <-- Check the indentation of this line
    def __init__(self, parent, go_next):
        self.parent = parent
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        ttk.Label(self.frame, text="Welcome to Ashcroft_Johnson's Stroke Diagnosis and Prediction Dashboard",
                  font=("Helvetica", 18, "bold"), background="light blue").pack(pady=20)

        ttk.Label(self.frame, text="VERIFICATION PAGE",
                  font=("Helvetica", 14, "bold"), background="light blue").pack()
        
        self.checkbox_var = tk.BooleanVar(value=False)
        self.login_button = ttk.Button(self.frame, text="LOGIN", command=go_next, state="disabled", style="Green.TButton")
        self.login_button.pack(side="left", padx=10)

        # Added this command@14:12 by 18/08/2023
        # Use self.go_to_registration_page as the command for the REGISTER button
        self.register_button = ttk.Button(self.frame, text="REGISTER", command=self.go_to_registration_page, style="Green.TButton")
        self.register_button.pack(side="right", padx=10)
        
        ttk.Label(self.frame, text="New User? Register to use the application").pack()

        ttk.Checkbutton(self.frame, text="Already have an account, please login", variable=self.checkbox_var,
                        command=self.update_login_button_state).pack()

    def update_login_button_state(self):
        if self.checkbox_var.get():
            self.login_button.state(["!disabled"])
        else:
            self.login_button.state(["disabled"])
            
    def go_to_registration_page(self):
        self.frame.destroy()  # Destroy the current frame
        RegistrationPage(self.parent, self.go_next, self.go_to_registration_page)  # Create a RegistrationPage instance

    # Other methods...
# added just now 
    def go_next(self):
        # Implement the logic to proceed to the next page
        pass
        
        
        
# LOGIN PAGE OKAY, Works Well by 18/08/2023 by 23:43
class LoginPage:
    def __init__(self, parent, go_next):
        self.parent = parent
        self.go_next = go_next
        self.attempts = 0  # Initialise the attempts counter
        
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        ttk.Label(self.frame, text="Ashcroft_Johnson's Stroke Diagnosis and Prediction Dashboard",
                  font=("Helvetica", 18, "bold"), background="light blue").pack(pady=20)
        ttk.Label(self.frame, text="Login Page", font=("Helvetica", 24, "bold")).pack(pady=(50, 20))
        
        ttk.Label(self.frame, text="Email Address:", font=("Helvetica", 12)).pack()
        self.username_entry = ttk.Entry(self.frame, font=("Helvetica", 12))
        self.username_entry.pack(padx=20, pady=5, fill="x")
        
        ttk.Label(self.frame, text="Password:", font=("Helvetica", 12)).pack()
        self.password_entry = ttk.Entry(self.frame, show="*", font=("Helvetica", 12))
        self.password_entry.pack(padx=20, pady=5, fill="x")
        
        self.checkbox_var = tk.BooleanVar(value=False)
        ttk.Checkbutton(self.frame, text="Remember me", variable=self.checkbox_var).pack(pady=(10, 20))
        
        self.login_button = ttk.Button(self.frame, text="Login", command=self.perform_login, style="Green.TButton")
        self.login_button.pack(pady=(0, 20), padx=20, fill="x")
        
        ttk.Label(self.frame, text="New User? Register to use the application", font=("Helvetica", 10)).pack()

        self.status_label = ttk.Label(self.frame, text="", font=("Helvetica", 12))
        self.status_label.pack()
        
        self.forgot_password_label = ttk.Label(self.frame, text="Forgot Password?", foreground="blue", cursor="hand2", font=("Helvetica", 12))
        self.forgot_password_label.pack()
        self.forgot_password_label.bind("<Button-1>", self.open_forgot_password_link)
        
        self.forgot_email_label = ttk.Label(self.frame, text="Forgot Email Address?", foreground="blue", cursor="hand2", font=("Helvetica", 12))
        self.forgot_email_label.pack()
        self.forgot_email_label.bind("<Button-1>", self.open_forgot_email_link)

    def open_forgot_email_link(self, event):
        self.parent.withdraw()  # Hide the main window
        self.forgot_email_window = tk.Toplevel()
        self.forgot_email_window.title("Forgot Email Address Recovery")
        
        ttk.Label(self.forgot_email_window, text="Forgot Email Address Recovery", font=("Helvetica", 16, "bold")).pack(pady=20)
        
        ttk.Label(self.forgot_email_window, text="Please contact the system administrator to recover your email address.",
                  font=("Helvetica", 12)).pack(pady=20)
        
        ttk.Button(self.forgot_email_window, text="Close", command=self.close_forgot_email_window).pack(pady=10)
        
    def close_forgot_email_window(self):
        self.forgot_email_window.destroy()
        self.parent.deiconify()  # Show the main window again
        

    def perform_login(self):
        email = self.username_entry.get()
        password = self.password_entry.get()
        
        if email == "admin" and password == "password":
            self.status_label.config(text="Login successful", foreground="green")
            self.attempts = 0  # Reset the attempts counter on successful login
        else:
            self.attempts += 1
            self.status_label.config(text="Login failed", foreground="red")
            
            if self.attempts >= 3:
                self.block_account()
                
    def block_account(self):
        messagebox.showerror("Account Blocked", "You have entered incorrect credentials for the third time.\nYour account is blocked; Contact the Administrator.")
        self.parent.quit()  # Quit the application after blocking the account

            
    def open_forgot_password_link(self, event):
        email = simpledialog.askstring("Forgot Password", "Enter your email address:")
        if email:
            if self.send_password_reset_email(email):
                messagebox.showinfo("Forgot Password", f"Password reset link has been sent to {email}")
            else:
                messagebox.showerror("Error", "Failed to send password reset to the email provided.")
        
    def send_password_reset_email(self, email):
        try:
            smtp_server = "smtp.gmail.com"
            smtp_port = 465
            sender_email = "ashcroft.j.johnson@gmail.com"
            sender_password = "cpwmuzivjihmzfhx"
            
            server = smtplib.SMTP_SSL(smtp_server, smtp_port)
            server.login(sender_email, sender_password)
            
            subject = "Password Reset"
            message = "Click the link to reset your password: https://email_address/reset_password"
            email_content = MIMEText(message)
            email_content["Subject"] = subject
            email_content["From"] = sender_email
            email_content["To"] = email
            
            server.sendmail(sender_email, [email], email_content.as_string())
            server.quit()
            return True
        
        except Exception as e:
            print("Error sending email:", e)
            return False

        
 # REGISTRATION PAGE Works Well @18/08/2023 by  23:43   
class RegistrationPage:
    def __init__(self, parent, go_next, go_back):
        self.parent = parent
        self.go_next = go_next
        self.go_back = go_back
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)

        ttk.Label(self.frame, text="Ashcroft_Johnson's Stroke Diagnosis and Prediction Dashboard",
               font=("Helvetica", 14, "bold"), background="light blue").pack(pady=10)
        ttk.Label(self.frame, text="Registration Page",
                  font=("Helvetica", 14, "bold")).pack(pady=(10, 5))
        
        self.username_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.nhs_number_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.full_name_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.age_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.gender_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.email_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.mobile_entry = ttk.Entry(self.frame, font=("Helvetica", 10))
        self.password_entry = ttk.Entry(self.frame, show="*", font=("Helvetica", 10))
        self.re_password_entry = ttk.Entry(self.frame, show="*", font=("Helvetica", 10))
        
        labels_and_entries = [
            ("Username:", self.username_entry),
            ("Enter Your NHS Confidential Number:", self.nhs_number_entry),
            ("Your Full Name:", self.full_name_entry),
            ("Age:", self.age_entry),
            ("Gender:", self.gender_entry),
            ("Email Address:", self.email_entry),
            ("Enter Your Mobile Number:", self.mobile_entry),
            ("Create Password:", self.password_entry),
            ("Re-enter Password:", self.re_password_entry)
        ]
        
        for label_text, entry_widget in labels_and_entries:
            label = ttk.Label(self.frame, text=label_text, font=("Helvetica", 12), anchor="w")
            label.pack(padx=10, pady=2, fill="x")
            entry_widget.pack(padx=10, pady=2, fill="x")
            
        ttk.Button(self.frame, text="Create Account", command=self.create_account, style="Green.TButton").pack(pady=10, padx=10, fill="x")
        ttk.Button(self.frame, text="Back", command=self.go_back).pack(pady=5, padx=10, fill="x")

    def create_account(self):
        # Implement the logic to create the user account
        # After creating the account, show a popup message
        messagebox.showinfo("Account Created", "Your account has been successfully created!")
        
        # Navigate to the Login Page
        self.parent.withdraw()  # Hide the current window
        login_page = LoginPage(self.parent, self.go_next)
        
        
class DataProtectionPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)

        # Design your Data Protection Page here

class HomePage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)

        
class SystemAdministratorAuthorisationPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class UsersPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class PredictionPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class AnalysisPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class SystemAdminEditingPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
        
class ContactUsPage:
    def __init__(self, parent, go_next):
        self.frame = ttk.Frame(parent)
        self.frame.pack(fill="both", expand=True)
        
class StrokePredictionDashboard:
    def __init__(self, root):
        self.root = root
        self.root.title("Stroke Diagnosis and Prediction Dashboard")

        self.pages = [
            VerificationPage, LoginPage, RegistrationPage, 
            DataProtectionPage, HomePage, SystemAdministratorAuthorisationPage,
            UsersPage, PredictionPage, AnalysisPage,SystemAdminEditingPage,
            ContactUsPage# Add other pages here
        ]

        self.current_page_index = 0

        self.back_button = ttk.Button(root, text="Back", command=self.go_back)
        self.update_buttons()

        self.show_page(self.current_page_index)

    def update_buttons(self):
        if self.current_page_index == 0:
            self.back_button.pack_forget()
        else:
            self.back_button.pack(side="left")

    def show_page(self, index):
        if 0 <= index < len(self.pages):
            if hasattr(self, "current_page"):
                self.current_page.frame.pack_forget()

            self.current_page = self.pages[index](self.root, self.go_next)
            self.update_buttons()

    def go_back(self):
        if self.current_page_index > 0:
            self.current_page_index -= 1
            self.show_page(self.current_page_index)

    def go_next(self):
        if self.current_page_index < len(self.pages) - 1:
            self.current_page_index += 1
            self.show_page(self.current_page_index)

if __name__ == "__main__":
    root = tk.Tk()
    
     # Added now 18/08/2023
    # Create a custom style
    style = ttk.Style(root)
    style.configure("Green.TButton", background="green")
    
    
     # Set the screen size to 13 inches (adjust the width and height values as needed)
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    inches_per_mm = 0.0393701
    desired_screen_size_inches = 20
    desired_width = int(screen_width * inches_per_mm * desired_screen_size_inches)
    desired_height = int(screen_height * inches_per_mm * desired_screen_size_inches)
    root.geometry(f"{desired_width}x{desired_height}")
    
    app = StrokePredictionDashboard(root)
    root.mainloop()